In [1]:
import ast
import os

In [16]:
with open('setup-test.py', 'r') as f:
    tree = ast.parse(f.read())
ast.dump(tree)

'Module(body=[], type_ignores=[])'

In [7]:
# read credentials from config
import configparser

config = configparser.ConfigParser()
if (os.path.exists('config.ini')):
    config.read('config.ini')
else:
    raise Exception('config.ini not found')
    
try:
    github_proxy=config['Proxy']['proxy']
except KeyError:
    github_proxy= None

In [30]:
# # parse setup.py with ast (deprecated)
# # developers can do literally anything in setup.py
# # makes it hard to parse setup.py

# import textwrap

# def parse_setup(setup_filename):
#     """Parse setup.py and return args and keywords args to its setup
#     function call

#     """
#     mock_setup = textwrap.dedent('''\
#     def setup(*args, **kwargs):
#         __setup_calls__.append((args, kwargs))
#     ''')
#     parsed_mock_setup = ast.parse(mock_setup, filename=setup_filename)
#     with open(setup_filename, 'rt') as setup_file:
#         parsed = ast.parse(setup_file.read())
#         for index, node in enumerate(parsed.body[:]):
#             if (
#                 not isinstance(node, ast.Expr) or
#                 not isinstance(node.value, ast.Call) 
#                 node.value.func.name != 'setup'
#             ):
#                 continue
#             print(node.value)
#             parsed.body[index:index] = parsed_mock_setup.body
#             break

#     fixed = ast.fix_missing_locations(parsed)
#     codeobj = compile(fixed, setup_filename, 'exec')
#     local_vars = {}
#     global_vars = {'__setup_calls__': []}
#     exec(codeobj, global_vars, local_vars)
    
#     # no setup calls
#     if not global_vars['__setup_calls__']:
#         return []
#     else:
#         return global_vars['__setup_calls__'][0]

# parse_setup('setup-test.py')

In [ ]:
REPOS_DIR = 'repos/'
if not os.path.exists(REPOS_DIR):
    os.mkdir(REPOS_DIR)

In [15]:
import os
from distutils.core import run_setup
from utils.download_from_github import download_file_from_github

def setup_runner(owner: str, repo: str, branch='master', setup_file='setup.py', REPOS_DIR='repos/', max_tries=3):

    path = os.path.abspath(f'{REPOS_DIR}/{owner}/{repo}/{branch}/')
    cwd = os.getcwd()
    
    result = None
    
    for _ in range(max_tries):
        try:
            os.chdir(path)
            result = run_setup(setup_file, stop_after="init")
            break
        except FileNotFoundError as e:
            os.chdir(cwd)
            # download on demand
            print(f'Missing {owner}/{repo}/{e.filename}, downloading')
            download_file_from_github(owner=owner, repo=repo, branch=branch, file=e.filename, proxy=github_proxy)
    
    os.chdir(cwd)
    
    return result.install_requires if result else None

setup_runner('hall-lab','svtyper')

Missing hall-lab/svtyper/setup.py, downloading
Missing hall-lab/svtyper/README.md, downloading
Missing hall-lab/svtyper/svtyper/version.py, downloading


FileNotFoundError: [Errno 2] No such file or directory: 'repos//hall-lab/svtyper/master/svtyper/version.py'

FileNotFoundError: [Errno 2] No such file or directory: 'README.md'